In [1]:
!pip install --quiet sklearn

In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from IPython.display import clear_output

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [77]:
df_train, df_test = train_test_split(pd.read_csv('https://raw.githubusercontent.com/afonsir/cyrela-etl/main/data/parsed-data.csv'))

y_train = df_train.pop('p_dias_atraso_category')
y_test = df_test.pop('p_dias_atraso_category')

In [70]:
df_train.head()

,empresa,p_empresa,marca,p_marca,obra,p_obra,bloco,p_bloco,unidade,p_unidade,dt_venda,p_dt_venda_day,p_dt_venda_month,p_dt_venda_year,dt_chaves,p_dt_chaves_day,p_dt_chaves_month,p_dt_chaves_year,carteira_sd_gerencial,p_carteira_sd_gerencial,saldo_devedor,p_saldo_devedor,p_data_base_day,p_data_base_month,p_data_base_year,dias_atraso,p_dias_atraso,valor_pago_atualizado,p_valor_pago_atualizado,valor_pago,p_valor_pago,vgv,p_vgv
764,2044,0.646018,VIVAZ,3,9919,0.998892,1,1.0,1012,0.439236,2021-02-18,0.600000,0.166667,1.0105,2023-10-01,0.033333,0.833333,1.0115,211568,0.016274,211567.63,0.016274,1.0,0.333333,1.0105,-7,0.006045,7121.70,0.000640,7000.00,0.000636,218567.63,0.010662
2795,168,0.053097,LIVING,2,4718,0.475126,1,1.0,171,0.074219,2020-12-14,0.466667,1.000000,1.0100,2023-06-01,0.033333,0.500000,1.0115,551722,0.042440,551722.26,0.042440,1.0,0.333333,1.0105,-41,0.035406,665768.35,0.059851,654314.62,0.059483,1206036.88,0.058831
836,1166,0.368521,VIVAZ,3,9835,0.990433,1,1.0,312,0.135417,2019-12-31,1.033333,1.000000,1.0095,2022-05-01,0.033333,0.416667,1.0110,86596,0.006661,86595.61,0.006661,1.0,0.333333,1.0105,-20,0.017271,114378.52,0.010282,104732.08,0.009521,191327.69,0.009333
1810,2047,0.646966,CYRELA,1,7821,0.787613,1,1.0,176,0.076389,2017-11-29,0.966667,0.916667,1.0085,2021-03-01,0.033333,0.250000,1.0105,607796,0.046754,607795.50,0.046753,1.0,0.333333,1.0105,-62,0.053541,446580.92,0.040147,395785.82,0.035981,1003581.32,0.048955
980,3091,0.976928,VIVAZ,3,8520,0.858006,1,1.0,907,0.393663,2021-04-09,0.300000,0.333333,1.0105,2023-07-01,0.033333,0.583333,1.0115,186299,0.014331,186298.86,0.014331,1.0,0.333333,1.0105,-39,0.033679,100.00,0.000009,100.00,0.000009,186398.86,0.009093


In [71]:
df_train.describe()

,empresa,p_empresa,p_marca,obra,p_obra,bloco,p_bloco,unidade,p_unidade,p_dt_venda_day,p_dt_venda_month,p_dt_venda_year,p_dt_chaves_day,p_dt_chaves_month,p_dt_chaves_year,carteira_sd_gerencial,p_carteira_sd_gerencial,saldo_devedor,p_saldo_devedor,p_data_base_day,p_data_base_month,p_data_base_year,dias_atraso,p_dias_atraso,valor_pago_atualizado,p_valor_pago_atualizado,valor_pago,p_valor_pago,vgv,p_vgv
count,2248.000000,2248.000000,2248.000000,2248.000000,2248.000000,2248.0,2248.0,2248.000000,2248.000000,2248.000000,2248.000000,2248.000000,2.248000e+03,2248.000000,2248.000000,2.248000e+03,2248.000000,2.248000e+03,2248.000000,2248.0,2.248000e+03,2.248000e+03,2248.000000,2248.000000,2.248000e+03,2248.000000,2.248000e+03,2248.000000,2.248000e+03,2248.000000
mean,1769.160587,0.559153,1.687278,7179.397242,0.723001,1.0,1.0,441.498221,0.191622,0.786329,0.602647,1.009750,3.333333e-02,0.577810,1.011125,6.343614e+05,0.048797,6.356840e+05,0.048899,1.0,3.333333e-01,1.010500e+00,-79.015125,0.068234,3.194264e+05,0.028716,2.868972e+05,0.026082,9.225812e+05,0.045004
std,848.590056,0.268202,0.753912,2443.205162,0.246043,0.0,0.0,504.181722,0.218829,0.254617,0.297866,0.000503,7.495673e-16,0.273135,0.000405,7.853763e+05,0.060414,7.786877e+05,0.059899,0.0,6.829391e-15,2.176521e-14,156.379659,0.135043,5.867247e+05,0.052745,5.204322e+05,0.047312,1.092420e+06,0.053289
min,164.000000,0.051833,1.000000,2339.000000,0.235549,1.0,1.0,1.000000,0.000434,0.100000,0.083333,1.008500,3.333333e-02,0.083333,1.009500,2.160000e+02,0.000017,2.160800e+02,0.000017,1.0,3.333333e-01,1.010500e+00,-1158.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,1.354457e+05,0.006607
25%,1166.000000,0.368521,1.000000,7470.000000,0.752266,1.0,1.0,103.000000,0.044705,0.600000,0.333333,1.009500,3.333333e-02,0.416667,1.011000,2.232792e+05,0.017175,2.239548e+05,0.017227,1.0,3.333333e-01,1.010500e+00,-52.000000,0.017271,6.008811e+04,0.005402,5.613009e+04,0.005103,3.635632e+05,0.017735
50%,2043.000000,0.645702,2.000000,7869.000000,0.792447,1.0,1.0,191.000000,0.082899,0.900000,0.666667,1.010000,3.333333e-02,0.583333,1.011000,4.112200e+05,0.031632,4.112453e+05,0.031634,1.0,3.333333e-01,1.010500e+00,-32.000000,0.027634,1.600377e+05,0.014387,1.473842e+05,0.013399,5.938666e+05,0.028969
75%,2154.000000,0.680784,2.000000,8887.000000,0.894965,1.0,1.0,704.250000,0.305664,1.000000,0.916667,1.010000,3.333333e-02,0.833333,1.011500,7.718065e+05,0.059370,7.766813e+05,0.059745,1.0,3.333333e-01,1.010500e+00,-20.000000,0.044905,3.698129e+05,0.033245,3.344557e+05,0.030405,1.190949e+06,0.058095
max,3164.000000,1.000000,3.000000,9930.000000,1.000000,1.0,1.0,2304.000000,1.000000,1.033333,1.000000,1.010500,3.333333e-02,1.000000,1.012000,1.300000e+07,1.000000,1.300000e+07,1.000000,1.0,3.333333e-01,1.010500e+00,0.000000,1.000000,1.112376e+07,1.000000,1.100000e+07,1.000000,2.050000e+07,1.000000


In [79]:
y_train.value_counts()

0    965
1    900
2    383
Name: p_dias_atraso_category, dtype: int64

In [72]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [ ]:
NUMERIC_COLUMNS = ['p_empresa', 'p_marca', 'p_obra', 'p_bloco', 'p_unidade',
                   'p_carteira_sd_gerencial', 'p_saldo_devedor', 'p_dias_atraso',
                   'p_valor_pago_atualizado', 'p_valor_pago', 'p_vgv',
                   'p_dt_venda_day', 'p_dt_venda_month', 'p_dt_venda_year',
                   'p_dt_chaves_day', 'p_dt_chaves_month', 'p_dt_chaves_year',
                   'p_data_base_day', 'p_data_base_month', 'p_data_base_year']

feature_columns = []

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[20, 20],
    n_classes=3)

In [ ]:
classifier.train(
    input_fn=lambda: input_fn(df_train, y_train, training=True),
    steps=75000)

In [80]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(df_test, y_test, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-06-15T00:48:02
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp14sey7lk/model.ckpt-75000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.35805s
INFO:tensorflow:Finished evaluation at 2021-06-15-00:48:02
INFO:tensorflow:Saving dict for global step 75000: accuracy = 0.636, average_loss = 0.69070786, global_step = 75000, loss = 0.6911561
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 75000: /tmp/tmp14sey7lk/model.ckpt-75000

Test set accuracy: 0.636

